In [2]:
!pip3 install Cython numpy
!pip3 install git+https://github.com/lopuhin/python-adagram.git

  Using cached https://files.pythonhosted.org/packages/d1/54/9d66ee2180776dfe33c2b8cc2f2b67c343fd9d80de91ac0edc5bc346fb06/Cython-0.29.15-cp36-cp36m-manylinux1_x86_64.whl
  Using cached https://files.pythonhosted.org/packages/62/20/4d43e141b5bc426ba38274933ef8e76e85c7adea2c321ecf9ebf7421cedf/numpy-1.18.1-cp36-cp36m-manylinux1_x86_64.whl
  Cloning https://github.com/lopuhin/python-adagram.git to /tmp/pip-mf24uhyi-build
  Using cached https://files.pythonhosted.org/packages/d1/54/9d66ee2180776dfe33c2b8cc2f2b67c343fd9d80de91ac0edc5bc346fb06/Cython-0.29.15-cp36-cp36m-manylinux1_x86_64.whl
  Using cached https://files.pythonhosted.org/packages/28/5c/cf6a2b65a321c4a209efcdf64c2689efae2cb62661f8f6f4bb28547cf1bf/joblib-0.14.1-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/62/20/4d43e141b5bc426ba38274933ef8e76e85c7adea2c321ecf9ebf7421cedf/numpy-1.18.1-cp36-cp36m-manylinux1_x86_64.whl
  Using cached https://files.pythonhosted.org/packages/65/eb/1f97cb97bfc2390a276969c

In [3]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /home/dima/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [4]:
import adagram
import pandas as pd
from lxml import html
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.corpus import wordnet as wn
from pymorphy2 import MorphAnalyzer
from string import punctuation
import json, os
from collections import Counter
import numpy as np
from matplotlib import pyplot as plt
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')
morph = MorphAnalyzer()
punct = punctuation+'«»—…“”*№–'
stops = set(stopwords.words('russian'))

In [5]:

def tokenize(text):
    
    words = [word.strip(punct) for word in text.lower().split() if word and word not in stops]
    words = [word for word in words if word]

    return words

def normalize(text):
    
    words = tokenize(text)
    words = [morph.parse(word)[0].normal_form for word in words if word]

    return words

In [15]:
corpus = open('corpus_ng.txt').read()[:500000]
print(len(corpus))

500000


In [16]:
corpus = normalize(corpus)

In [17]:
f = open('corpus.txt', 'w')
f.write(' '.join(corpus))
f.close()

In [18]:
!adagram-train corpus.txt out.pkl

[INFO] 2020-03-16 01:11:59,873 Building dictionary...
[INFO] 2020-03-16 01:11:59,950 Done! 1032 words.
[INFO] 2020-03-16 01:12:02,793 100.00% -6.5942 0.0000 1.8/2.0 10.78 kwords/sec


In [19]:
vm = adagram.VectorModel.load("out.pkl")

In [20]:
vm.disambiguate("мир", ['чемпионат', "россия", "игра", "футбол"])

array([0.99851649, 0.00148351, 0.        , 0.        , 0.        ])

In [21]:
v = vm.sense_vector("мир", 0)

In [22]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

In [23]:

corpus_xml = html.fromstring(open('paraphraser/paraphrases.xml', 'rb').read())
texts_1 = []
texts_2 = []
classes = []

for p in corpus_xml.xpath('//paraphrase'):
    texts_1.append(p.xpath('./value[@name="text_1"]/text()')[0])
    texts_2.append(p.xpath('./value[@name="text_2"]/text()')[0])
    classes.append(p.xpath('./value[@name="class"]/text()')[0])
    
df_data = pd.DataFrame({'text_1':texts_1, 'text_2':texts_2, 'label':classes})

In [24]:
df_data['text_1_norm'] = df_data['text_1'].apply(normalize)
df_data['text_2_norm'] = df_data['text_2'].apply(normalize)

In [26]:
f = open('my_text.txt', 'w')
for i in df_data['text_1_norm']:
    f.write(' '.join(i))
for i in df_data['text_2_norm']:
    f.write(' '.join(i))
f.close()

In [27]:
!adagram-train my_text.txt out.pkl

[INFO] 2020-03-16 01:13:22,006 Building dictionary...
[INFO] 2020-03-16 01:13:22,202 Done! 1358 words.
[INFO] 2020-03-16 01:13:28,966 100.00% -6.6400 0.0000 1.7/3.0 10.26 kwords/sec


In [28]:
vm = adagram.VectorModel.load("out.pkl")

In [29]:
words = [1,2,3,4,5,6,7,8,9]
#Функция из Задания 1
def get_words_in_context(words, window=3):
  main_list = []
  for i, word in enumerate(words):
    local_list = []
    if i < window:
      local_list = [word, words[:window+i+1]]
      local_list[1].remove(word)
      main_list.append(local_list)
    else:           
      local_list = [word, words[i - window:i + window+1]]
      local_list[1].remove(word)
      main_list.append(local_list)

  return main_list

In [30]:
get_words_in_context(words, window=3)

[[1, [2, 3, 4]],
 [2, [1, 3, 4, 5]],
 [3, [1, 2, 4, 5, 6]],
 [4, [1, 2, 3, 5, 6, 7]],
 [5, [2, 3, 4, 6, 7, 8]],
 [6, [3, 4, 5, 7, 8, 9]],
 [7, [4, 5, 6, 8, 9]],
 [8, [5, 6, 7, 9]],
 [9, [6, 7, 8]]]

In [31]:
#get_embedding для адаграма
def get_embedding_adagram(text, model, window, dim):   
    
    word2context = get_words_in_context(text, window)
    
    
    vectors = np.zeros((len(word2context), dim))
    
    for i, (word, context) in enumerate(word2context):
        
        try:
            v = vm.disambiguate(word, context).argmax()
            vectors[i] = model.sense_vector(word, v)
        
        except (KeyError, ValueError) as e:
            continue
    
    if vectors.any():
        vector = np.average(vectors, axis=0)
    else:
        vector = np.zeros((dim))
    
    return vector

In [32]:

df_data['text_1_notnorm'] = df_data['text_1'].apply(tokenize)
df_data['text_2_notnorm'] = df_data['text_2'].apply(tokenize)

X_text_1_ft = np.zeros((len(df_data['text_1_norm']), 100))
X_text_2_ft = np.zeros((len(df_data['text_2_norm']), 100))

for i, text in enumerate(df_data['text_1_norm'].values):
    X_text_1_ft[i] = get_embedding_adagram(text, vm, 3, 100)
    
for i, text in enumerate(df_data['text_2_norm'].values):
    X_text_2_ft[i] = get_embedding_adagram(text, vm, 3, 100)

In [33]:
X_text_ft = np.concatenate([X_text_1_ft, X_text_2_ft], axis=1)

In [34]:
y = df_data['label'].values
train_X, valid_X, train_y, valid_y = train_test_split(X_text_ft, y,random_state=1)
clf = RandomForestClassifier(n_estimators=100, max_depth=7, min_samples_leaf=15,
                             class_weight='balanced')
clf.fit(train_X, train_y)
preds = clf.predict(valid_X)
print(classification_report(valid_y, preds))

              precision    recall  f1-score   support

          -1       0.59      0.53      0.56       629
           0       0.51      0.21      0.29       737
           1       0.33      0.70      0.45       441

    accuracy                           0.44      1807
   macro avg       0.47      0.48      0.43      1807
weighted avg       0.49      0.44      0.42      1807



In [35]:
#оценка кросс-валидацией
from sklearn.model_selection import cross_val_score
scores = cross_val_score(clf, X_text_ft, y, cv=5, scoring = 'f1_micro')
scores.mean()

0.3915825544276778

In [106]:
import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /home/dima/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

In [107]:
len(list(wn.all_synsets()))

117659

In [120]:
word = 'day'
for synset in wn.synsets(word):
    print(synset.definition().split())

['time', 'for', 'Earth', 'to', 'make', 'a', 'complete', 'rotation', 'on', 'its', 'axis']
['some', 'point', 'or', 'period', 'in', 'time']
['a', 'day', 'assigned', 'to', 'a', 'particular', 'purpose', 'or', 'observance']
['the', 'time', 'after', 'sunrise', 'and', 'before', 'sunset', 'while', 'it', 'is', 'light', 'outside']
['the', 'recurring', 'hours', 'when', 'you', 'are', 'not', 'sleeping', '(especially', 'those', 'when', 'you', 'are', 'working)']
['an', 'era', 'of', 'existence', 'or', 'influence']
['the', 'period', 'of', 'time', 'taken', 'by', 'a', 'particular', 'planet', '(e.g.', 'Mars)', 'to', 'make', 'a', 'complete', 'rotation', 'on', 'its', 'axis']
['the', 'time', 'for', 'one', 'complete', 'rotation', 'of', 'the', 'earth', 'relative', 'to', 'a', 'particular', 'star,', 'about', '4', 'minutes', 'shorter', 'than', 'a', 'mean', 'solar', 'day']
['a', 'period', 'of', 'opportunity']
['United', 'States', 'writer', 'best', 'known', 'for', 'his', 'autobiographical', 'works', '(1874-1935)']


In [109]:
sent = "this is a good to die"

In [246]:
#алгоритм Леска
def lesk( word, sentence_dict):
    bestsense = 0
    maxoverlap = 0
    sentence_set = set(normalize(" ".join(sentence_dict)))
    for i, syns in enumerate(wn.synsets(word)):
        syns = normalize(syns.definition())
        if len(sentence_set & set(syns)) > maxoverlap:
            maxoverlap = len(sentence_set & set(syns))
            bestsense = i
    return (wn.synsets(word)[bestsense])


In [236]:
lesk('day', sent.split()).definition()

2
10


'a day assigned to a particular purpose or observance'

In [146]:
#проверка на реальном корпусе
corpus_wsd = []
corpus = open('corpus_wsd_50k.txt').read().split('\n\n')[:1000]
for sent in corpus:
    corpus_wsd.append([s.split('\t') for s in sent.split('\n')])

In [186]:
from collections import Counter

In [197]:
wn.lemma_from_key('long%3:00:02::').synset()

Synset('long.a.01')

In [247]:
res_list = []
lebel_list = []
for sent_list in corpus_wsd:
    my_sentence = []
    key_dict = Counter()
    for word_list in sent_list:
        if word_list[0]:
            key_dict[word_list[2]] = word_list[0]
        my_sentence.append(word_list[2])
    for word in my_sentence:
        if key_dict[word]:
            try:
                if lesk(word, my_sentence) == wn.lemma_from_key(key_dict[word]).synset():
                    res_list.append(1)
                else:
                    res_list.append(0)
            except IndexError:
                res_list.append(0)
            lebel_list.append(1)
                
                

In [248]:
from sklearn.metrics import accuracy_score

In [249]:
accuracy_score(res_list, lebel_list, normalize=True)

0.3167509191176471